In [22]:
import pymatgen as pmg
from pymatgen.io.cif import CifParser
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

import lammps

In [3]:
structure = CifParser('../data/structure/MgO.cif').get_structures()[0]
structure.remove_oxidation_states()
spga = SpacegroupAnalyzer(structure)
conventional_structure = spga.get_conventional_standard_structure() * (5, 5, 5)

In [4]:
print('num atoms', len(conventional_structure))

num atoms 1000


In [23]:
elements = pmg.Element('Mg'), pmg.Element('O') # specify index for each element
mg, o = elements

buckingham_potential_parameters = {
    (mg, mg): [1309362.2766468062, 0.104, 0.0],
    (mg, o): [9892.357, 0.20199, 0.0],
    (o, o): [2145.7345, 0.3, 30.2222]
}

charges = {
    mg: 1.4,
    o: -1.4
}

In [68]:
import numpy as np
import functools

def buckingham_potential(r, A, p, C):
    return A * np.exp(-r/p) - C / r**6

In [69]:
f = functools.partial(buckingham_potential, A=1, p=2, C=3)
f(1.0)

-2.393469340287367

In [70]:
def write_function_pair_potential(func, dfunc=None, bounds=(1.0, 10.0), samples=500, toll=1e-6, keyword='PAIR', filename='lammps.table'):
    r_min, r_max = bounds
    if dfunc is None:
        dfunc = lambda r: (func(r+toll) - func(r-toll)) / (2*toll)

    with open(filename, 'w') as f:
        i = np.arange(1, n+1)
        r = np.linspace(r_min, r_max, n)
        forces = func(r)
        energies = dfunc(r)
        #lines = ('%d %f %f %f\n' % (index, radius, force, energy) for index, radius, force, energy in zip(i, r, forces, energies))
        f.write("%s\nN %d\n\n" % (keyword, samples) + ''.join('%d %f %f %f\n' % (index, radius, force, energy) for index, radius, force, energy in zip(i, r, forces, energies)))


In [71]:
%%timeit

write_function_pair_potential(f)

3.89 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Traditional pair/overlay approach

In [47]:
lmp = lammps.Lammps(units='metal', style='full')
elements = lmp.system.add_pymatgen_structure(conventional_structure, elements)

lmp.command('kspace_style pppm 1e-6')
for c in charges:
    lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))

lmp.command('pair_style hybrid/overlay coul/long 10.0 buck 10.0')
lmp.command('pair_coeff * * coul/long')
for p in buckingham_potential_parameters:
    lmp.command('pair_coeff %d %d buck %f %f %f' % (elements.index(p[0])+1, elements.index(p[1])+1, *buckingham_potential_parameters[p]))    

In [48]:
%%timeit

lmp.run(0)

58.2 ms ± 495 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
print('potential energy', lmp.thermo.potential_energy.scalar)
print('temperature', lmp.thermo.temperature.scalar)
print('pressure', lmp.thermo.pressure.vector)

potential energy 1022.4233073482385
temperature 0.0
pressure [ 6.43097404e+05  6.43097404e+05  6.41907177e+05  5.50050186e-10
 -1.87660923e-09 -1.99158064e-09]


# pair/overlay + pair table

In [73]:
lmp = lammps.Lammps(units='metal', style='full')
elements = lmp.system.add_pymatgen_structure(conventional_structure, elements)

lmp.command('kspace_style pppm 1e-6')
for c in charges:
    lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))

samples = 2000
lmp.command('pair_style hybrid/overlay coul/long 10.0 table linear %d' % n)
lmp.command('pair_coeff * * coul/long')
for p in buckingham_potential_parameters:
    params = buckingham_potential_parameters[p]
    keyword = 'PAIR'
    filename = filename='%s-%s.table' % (p[0].symbol, p[1].symbol)
    f = functools.partial(buckingham_potential, A=params[0], p=params[1], C=params[2])
    write_function_pair_potential(f, filename=filename, keyword=keyword, samples=samples)
    lmp.command('pair_coeff %d %d table %s %s' % (elements.index(p[0])+1, elements.index(p[1])+1, filename, keyword))

In [74]:
%%timeit

lmp.run(0)

52 ms ± 895 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [75]:
print('potential energy', lmp.thermo.potential_energy.scalar)
print('temperature', lmp.thermo.temperature.scalar)
print('pressure', lmp.thermo.pressure.vector)

potential energy 1022.8495475347404
temperature 0.0
pressure [-6.43378919e+05 -6.43378919e+05 -6.44569145e+05 -2.63779993e-10
  2.12794654e-09  2.02697490e-09]
